<a href="https://colab.research.google.com/github/kapiloo8/Stock_prediction-using-svm-/blob/main/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

for machine learning we have to import

In [67]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

for data manipulation and ploting 

In [68]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
import warnings 
warnings.filterwarnings("ignore")


We will Read the Stock Data Downloaded From Yahoo Finance Website. The Data Is stored in OHLC(Open, High, Low, Close)  format in a CSV file. To read a CSV file, you can use the read_csv() method of pandas.

In [ ]:
df = pd.read_csv('/content/ADANIENT.NS.csv')
print(df)

The data needed to be processed before use such that the date column should act as an index to do that

In [ ]:
# Changes The Date column as index columns
df.index = pd.to_datetime(df['Date'])
df
# drop The original date column
df = df.drop(['Date'],axis='columns')
df

Explanatory or independent variables are used to predict the value response variable. The X is a dataset that holds the variables which are used for prediction. The X consists of variables such as ‘Open – Close’ and ‘High – Low’. These can be understood as indicators based on which the algorithm will predict tomorrow’s trend. Feel free to add more indicators and see the performance

In [ ]:
# Create predictor variables
df['open-close'] = df.Open - df.Close
df['high-low']= df.High - df.Low
# Store all predictor variables in a variable X
x = df[['open-close','high-low']]
x.head()


the target variable is the outcome which the machine learning model will predict based on the explanatory variables. y is a target dataset storing the correct trading signal which the machine learning algorithm will try to predict. If tomorrow’s price is greater than today’s price then we will buy the particular Stock else we will have no position in the. We will store +1 for a buy signal and 0 for a no position in y. We will use where() function from NumPy to do this.

In [ ]:
y = np.where(df['Close'].shift(-1)>df['Close'],1,0)
print(y)

We will split data into training and test data sets. This is done so that we can evaluate the effectiveness of the model in the test dataset

In [73]:
split_percentage = 0.8
split = int(split_percentage*len(df))
#train set data
X_train = x[:split]
Y_train = y[:split]
#test set data
X_test = x[split:]
Y_test = y[split:]

We will use SVC() function from sklearn.svm.SVC library to create our classifier model using the fit() method on the training data set.

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [0.01, 0.1, 1, 10]}

# Create an SVM model with a radial basis function (RBF) kernel
svc = SVC(kernel='rbf')

# Create a grid search object with cross-validation
grid_search = GridSearchCV(svc, param_grid, cv=5)

# Fit the grid search object to the training data
grid_search.fit(X_train, Y_train)

In [75]:
print("Best parameters: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)


Best parameters:  {'C': 10, 'gamma': 0.01}
Best cross-validation score:  0.5732051282051283


In [ ]:
best_svc = grid_search.best_estimator_
best_svc.fit(X_train, Y_train)

# Make predictions on the test data using the best SVM model
Y_pred = best_svc.predict(X_test)

# Evaluate the accuracy of the best SVM model on the test data
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy score: ", accuracy)

Classifier accuracy
We will compute the accuracy of the algorithm on the train and test the data set by comparing the actual values of the signal with the predicted values of the signal. The function accuracy_score() will be used to calculate the accuracy.

In [77]:
# Make predictions on the entire dataset using the best SVM model
df['Predicted_Signal'] = best_svc.predict(x)

In [78]:
# Calculate daily returns
df['Return'] = df.Close.pct_change()

In [79]:
# Calculate strategy returns
df['Strategy_Return'] = df.Return *df.Predicted_Signal.shift(1)

In [80]:
# Calculate Cumulative returns
df['Cum_Ret'] = df['Return'].cumsum()


In [81]:
# Plot Cumulative returns
df['Cum_Strategy'] = df['Strategy_Return'].cumsum()

In [ ]:
plt.plot(df['Cum_Ret'],color='red')
plt.plot(df['Cum_Strategy'],color='blue')
plt.show()